# Imports

In [2]:
from ultralytics import YOLO
import os
from azure.storage.blob import BlobServiceClient
from pydicom import dcmread
from io import BytesIO
import numpy as np
import cv2
import pylidc as pl

# Connect to Blob

In [3]:
with open('/home/andrew/ITRI-LungCancer/keys.txt', 'r') as file:
    data = file.read().splitlines()
    account_name    = data[0]
    account_key     = data[1]
    container_name  = data[2]

blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
container_client = blob_service_client.get_container_client(container_name)
blob_name_list = container_client.list_blob_names()

In [ ]:
blob_name = next(blob_name_list)
scan_name = blob_name.split('/')[0]
slice_num = blob_name.split('/')[3].split('-')[1]

blob_client = container_client.get_blob_client(blob_name)
blob_data = blob_client.download_blob().readall()
blob_stream = BytesIO(blob_data)
ds = dcmread(blob_stream)

scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == ds.PatientID).first()
slice_location = ds.ImagePositionPatient[2]

for ann_count, ann in enumerate(scan.annotations):
    for contour in ann.contours:
        if abs(contour.image_z_position - slice_location) < scan.slice_spacing:
            asd

In [ ]:
model = YOLO('/home/andrew/ITRI-LungCancer/YOLO/model.pt')

path = '/home/andrew/ITRI-LungCancer/sample/'
imgs = os.listdir(path)
imgs.sort()

model_input = []
for img in imgs:
    model_input.append(path+img)

results = model(model_input, conf=0.5, verbose=True)

result = results[0]
print(result.boxes)